## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

### Sequential LCEL Chain

In [9]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate
                                    )

In [3]:
base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(
    base_url=base_url,
    model=model
)

In [5]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate.from_messages(messages=messages)

query = template.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})

response = llm.invoke(query)

print(response.content)

Here's what I know:

* The Sun is the center of our solar system.
* There are eight planets: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
* Pluto was considered a planet until it was reclassified as a dwarf planet in 2006.
* The four major planets (Mercury to Neptune) have distinct characteristics - Earth is known for being the only known planet with life.
* The solar system also has other objects like moons, asteroids, and comets.


##### Lets create a chain for above

In [6]:
chain = template | llm

In [7]:
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's information about the solar system in 5 points:

• The sun is at the center of our solar system, providing energy and light to all planets.
• There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
• The four largest planets in our solar system are Jupiter, Saturn, Uranus, and Neptune - they are known as the gas giants.
• The five inner planets are Mercury, Venus, Earth, Mars, and Jupiter - they are rocky worlds with different environments.
• Pluto was previously considered a planet but is now classified as a dwarf planet by some astronomers.


In [8]:
response = chain.invoke({'school': 'PhD.', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's a brief overview of the solar system:

• The sun is at the center, and it's responsible for most solar energy on Earth.
• There are eight planets: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
• Pluto was once considered a planet but is now classified as a dwarf planet due to its small size.
• The four inner planets (Mercury, Mars, Venus, and Earth) orbit very close to the sun due to their rapid rotation periods.
• The outer planets (Jupiter, Saturn, Uranus, and Neptune) take much longer to complete one rotation around the sun due to their larger sizes.


##### StrOutputParser

In [10]:
chain = template | llm | StrOutputParser()

response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response)


Here's a brief overview of the solar system in 5 points:

* Our solar system consists of eight planets, dwarf planets, and other objects.
* The Sun is at the center, and it provides light and warmth to our solar system.
* The four inner planets are Mercury, Mars, Venus, and Earth - they're closest to the Sun.
* The four outer planets are Jupiter, Saturn, Uranus, and Neptune - they're farthest from the Sun.
* Pluto was once considered a planet but is now classified as a dwarf planet.
